In [42]:
## POST PROCESSING THE STATE FILES

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import sparse

## CODES FROM PROFESSOR FIDELKOWSKI

#-----------------------------------------------------------
# Identifies interior and boundary edges given element-to-node
# IE contains (n1, n2, elem1, elem2) for each interior edge
# BE contains (n1, n2, elem) for each boundary edge
def edgehash(E, B):
    Ne = E.shape[0]; Nn = np.amax(E)+1
    H = sparse.lil_matrix((Nn, Nn), dtype=np.int)
    IE = np.zeros([int(np.ceil(Ne*1.5)),4], dtype=np.int)
    ni = 0
    for e in range(Ne):
        for i in range(3):
            n1, n2 = E[e,i], E[e,(i+1)%3]
            if (H[n2,n1] == 0):
                H[n1,n2] = e+1
            else:
                eR = H[n2,n1]-1
                IE[ni,:] = n1, n2, e, eR
                H[n2,n1] = 0
                ni += 1
    IE = IE[0:ni,:]
    # boundaries
    nb0 = nb = 0
    for g in range(len(B)): nb0 += B[g].shape[0]
    BE = np.zeros([nb0,4], dtype=np.int)
    for g in range(len(B)):
        Bi = B[g]
        for b in range(Bi.shape[0]):
            n1, n2 = Bi[b,0], Bi[b,1]
            if (H[n1,n2] == 0): n1,n2 = n2,n1
            BE[nb,:] = n1, n2, H[n1,n2]-1, g
            nb += 1
    return IE, BE
#-----------------------------------------------------------

def readgri(fname):
    f = open(fname, 'r')
    Nn, Ne, dim = [int(s) for s in f.readline().split()]
    # read vertices
    V = np.array([[float(s) for s in f.readline().split()] for n in range(Nn)])
    # read boundaries
    NB = int(f.readline())
    B = []; Bname = []
    for i in range(NB):
        s = f.readline().split(); Nb = int(s[0]); Bname.append(s[2])
        Bi = np.array([[int(s)-1 for s in f.readline().split()] for n in range(Nb)])
        B.append(Bi)
    # read elements
    Ne0 = 0; E = []
    while (Ne0 < Ne):
        s = f.readline().split(); ne = int(s[0])
        Ei = np.array([[int(s)-1 for s in f.readline().split()] for n in range(ne)])
        E = Ei if (Ne0==0) else np.concatenate((E,Ei), axis=0)
        Ne0 += ne
            # make IE, BE structures
    IE, BE = edgehash(E, B)
    Mesh = {'V':V, 'E':E, 'IE':IE, 'BE':BE, 'Bname':Bname }
    return Mesh

#-----------------------------------------------------------
def readU(fname):
    return np.loadtxt(fname)
#-----------------------------------------------------------
def getField(U, field):
    r, ru, rv, rE = [U[:,i] for i in range(4)]
    g = 1.4
    V = np.sqrt(ru**2 + rv**2)/r
    p = (g-1.)*(rE-0.5*r*V**2)
    c = np.sqrt(g*p/r)
    M = V/c
    s = field.lower()
    if (s == 'mach'):
        return M
    else:
        return []
#-----------------------------------------------------------
def getnodestate(Mesh, U):
    V = Mesh['V']; E = Mesh['E']
    Nv, Ne = V.shape[0], E.shape[0]
    UN = np.zeros(Nv); count = np.zeros(Nv)
    for e in range(Ne):
        for i in range(3):
            n = E[e,i];
            UN[n] += U[e]; count[n] += 1
    UN /= count
    return UN

#-----------------------------------------------------------
def plotmesh(Mesh, fname):
    V = Mesh['V']; E = Mesh['E']; BE = Mesh['BE']
    f = plt.figure(figsize=(12,12))
    plt.triplot(V[:,0], V[:,1], E, 'k-')
    for i in range(BE.shape[0]):
        plt.plot(V[BE[i,0:2],0],V[BE[i,0:2],1], '-', linewidth=1, color='black')
    dosave = not not fname
    plt.axis('equal'); plt.axis('off')
    plt.axis([-0.5, 1.5,-1, 1])
    plt.tick_params(axis='both', labelsize=12)
    f.tight_layout();
    if (dosave): plt.savefig(fname)
    else: plt.show(block=True);
    plt.close(f)
    
#-----------------------------------------------------------
def plotstate(Mesh, U, p, field, frange, fname, ftitle):
    V = Mesh['V']; E = Mesh['E']; BE = Mesh['BE']
    f = plt.figure(figsize=(12,12))
    F = getField(U, field)
    if (p == 0):
        plt.tripcolor(V[:,0], V[:,1], triangles=E, facecolors=F, shading='flat')
    else:
        vc = np.linspace(frange[0], frange[1], 21) if (len(frange) > 0) else 20
        plt.tricontourf(V[:,0], V[:,1], E, getnodestate(Mesh,F), vc)
    for i in range(BE.shape[0]):
        plt.plot(V[BE[i,0:2],0],V[BE[i,0:2],1], '-', linewidth=2, color='black')
    dosave = not not fname
    plt.axis('equal'); plt.axis('off')
    if (len(frange)>0): plt.clim(frange[0], frange[1])
    plt.set_cmap('jet')
    cbar=plt.colorbar(orientation='horizontal', pad=-.12, fraction=.045)
    cbar.ax.tick_params(labelsize=16)
    plt.axis([-0.5, 1.5,-1, 1])
    plt.title(ftitle, fontsize=16)
    plt.tick_params(axis='both', labelsize=12)
    f.tight_layout();
    #if (dosave): plt.savefig(fname, bbox_inches='tight',pad_inches=-.2)
    #else: 
    plt.show(block=True);
    #plt.close(f)

In [83]:
def Mach_Plot(gri_file, state_file, title, fname):

    """ 
    Plots the Mach number for a given state file
    Inputs:
        gri_file: str, name of gri file
        state_file: str, name of state file solution
        title: str, if saving plot what you want the name to be
        fname: str, title of plot
    Output:
        plot
    """

    Mesh = readgri(gri_file)

    BC = Mesh['BE']
    V = Mesh['V']
    state = readU(state_file)

    field = getField(state, 'mach')
    plotstate(Mesh, state, 0, 'mach', [0, 1], fname, title)


    return


def cp(gri_file, state_file, uinf_file):

    """ 
    Calculates the pressure coefficient along airfoil elements
    Inputs:
        gri_file: str, name of gri file
        state_file: str, name of state file solution
        uinf_file: str, name of the file containing the solution at infinity
    Output for each element of airfoil:
        cp: array, pressure coefficient
        x1: array, where the pressure coefficient begins on the airfoil
        x2: array, where the pressure coefficient ends on the airfoil
    """

    CoarseMesh = readgri(gri_file)
    BC = CoarseMesh['BE']
    V = CoarseMesh['V']

    state = readU(state_file)

    uinf_file = pd.read_table(uinf_file, sep = '\s+', names=[1, 2, 3, 4])
    uinf = uinf_file.to_numpy()
    uinf = uinf[0]

    gamma = 1.4
    v_inf = (uinf[1]/uinf[0])**2 + (uinf[2]/uinf[0])**2
    p_inf = (gamma-1)*(uinf[3] - 0.5*uinf[0]*v_inf)
    rho_inf = uinf[0]

    # calculating cp for the slat airfoil
    slat_idx = np.where(BC[:,3] == 1)
    slat_elem = BC[slat_idx[0], 2]

    cp_slat = []
    x1_slat = V[BC[slat_idx[0], 0], 0]
    x2_slat = V[BC[slat_idx[0], 1], 0]
    for i in slat_elem:
        u = state[i]

        vx = u[1]/u[0]
        vy = u[2]/u[0]
        v_mag_sq = vx**2 + vy**2
        p = (gamma-1)*(u[3] - 0.5*u[0]*v_mag_sq)
        denom = 0.5*rho_inf*v_inf
            
        cp_slat.append((p-p_inf)/denom)

    # calculating cp for the main airfoil
    main_idx = np.where(BC[:,3] == 2)
    main_elem = BC[main_idx[0], 2]

    cp_main = []
    x1_main = V[BC[main_idx[0], 0], 0]
    x2_main = V[BC[main_idx[0], 1], 0]
    for i in main_elem:
        u = state[i]

        vx = u[1]/u[0]
        vy = u[2]/u[0]
        v_mag_sq = vx**2 + vy**2
        p = (gamma-1)*(u[3] - 0.5*u[0]*v_mag_sq)
        denom = 0.5*rho_inf*v_inf
            
        cp_main.append((p-p_inf)/denom)


    # calculating cp for the flap airfoil

    flap_idx = np.where(BC[:,3] == 3)
    flap_elem = BC[flap_idx[0], 2]

    cp_flap = []
    x1_flap = V[BC[flap_idx[0], 0], 0]
    x2_flap = V[BC[flap_idx[0], 1], 0]
    for i in flap_elem:
        u = state[i]

        vx = u[1]/u[0]
        vy = u[2]/u[0]
        v_mag_sq = vx**2 + vy**2
        p = (gamma-1)*(u[3] - 0.5*u[0]*v_mag_sq)
        denom = 0.5*rho_inf*v_inf
            
        cp_flap.append((p-p_inf)/denom)

    return cp_main, x1_main, x2_main, cp_slat, x1_slat, x2_slat, cp_flap, x1_flap, x2_flap


def lift_drag(gri_file, state_file, uinf_file, angle):

    """
    Calculates the life and draf coefficients for the entire airfoil 
    Inputs:
        gri_file: str, name of gri file
        state_file: str, name of state file solution
        uinf_file: str, name of the file containing the solution at infinity
    Output:
        Cl: float, lift coefficient
        Cd: float, drag coefficient
    """

    Mesh = readgri(gri_file)
    BC = Mesh['BE']
    print(BC)
    V = Mesh['V']

    state = readU(state_file)

    uinf_file_full = pd.read_table(uinf_file, sep = '\s+', names=[1, 2, 3, 4])
    uinf = uinf_file_full.to_numpy()
    uinf = uinf[0]

    all_idx = np.where( (BC[:,3] == 2) | (BC[:,3] == 1) | (BC[:,3] == 3))
    all_elem = BC[all_idx[0], 2]


    gamma = 1.4

    L = 0
    D = 0

    n1 = V[BC[all_idx[0], 0], :]
    n2 = V[BC[all_idx[0], 1], :]

    theta = np.deg2rad(angle)

    for i in range(len(all_elem)):
        u = state[all_elem[i]]
        length_cal = np.sqrt((n2[i, 0] - n1[i, 0])**2 + (n2[i, 1] - n1[i,1])**2)

        nx = (1/length_cal)*(n2[i,1] - n1[i,1])
        ny = (-(1/length_cal)*(n2[i,0] - n1[i,0]))

        vx = u[1]/u[0]
        vy = u[2]/u[0]
        v_mag_sq_L = vx**2 + vy**2
        pL = (gamma-1)*(u[3] - 0.5*u[0]*v_mag_sq_L)

        vx = u[1]/u[0]
        vy = u[2]/u[0]
        v_mag_sq_L = vx**2 + vy**2
        pD = (gamma-1)*(u[3] - 0.5*u[0]*v_mag_sq_L)    
        
        L += pL*nx*np.cos(theta + np.pi)*length_cal
        D += pD*ny*np.cos(theta)*length_cal


    v_inf = (uinf[1]/uinf[0])**2 + (uinf[2]/uinf[0])**2
    p_inf = (gamma-1)*(uinf[3] - 0.5*uinf[0]*v_inf)
    rho_inf = uinf[0]

    c_ref = 1                                                  # reference chord length value

    Cl = L/(0.5*rho_inf*v_inf*c_ref)
    Cd = D/(0.5*rho_inf*v_inf*c_ref)

    return Cl, Cd

def cp_plot(gri_file, state_file, uinf_file, fname):

    """ 
    Plots the pressure coefficient along airfoil elements
    Inputs:
        gri_file: str, name of gri file
        state_file: str, name of state file solution
        uinf_file: str, name of the file containing the solution at infinity
        fname: str, title of plot
    Output for each element of airfoil:
        plot
    """

    [cp_m, x1_m, x2_m, cp_s, x1_s, x2_s, cp_f, x1_f, x2_f] = cp(gri_file, state_file, uinf_file)

    # cp plotting
    plt.figure(figsize=(12,8))
    plt.hlines(cp_m, x1_m, x2_m, 'black', linewidth=2)
    plt.hlines(cp_s, x1_s, x2_s, 'black', linewidth=2)
    plt.hlines(cp_f, x1_f, x2_f, 'black', linewidth=2)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.legend(fontsize=12)
    plt.title(fname, fontsize=16)
    plt.xlabel('x', fontsize=14)
    plt.ylabel('-$c_p$', fontsize=14)
    plt.grid()
    plt.gca().invert_yaxis()

    return
